In [1]:
import os
import numpy as np
from sndaq.reader import SN_PayloadReader
from sndaq.writer import SN_PayloadWriter, construct_payload

#### Read one SN payload from existing file

In [2]:
data_dir = '../../data/sndata-spts-209058_001-010'
data_file = 'sn_209058_000001_239420_529095.dat'
with SN_PayloadReader(os.path.join(data_dir, data_file)) as r:
    pay = next(r)
print(pay)

Supernova@157144851073427008[dom 88ce18beda45 clk 03103a410001 scalerData*652


#### Custom Payload construction & Writing Payloads
Test `sndaq.writer.SN_PayloadWriter` and `sndaq.writer.construct_payloads` via the following steps
1. Write an existing SN payload to file
2. Construct a brand new custom SN payload
3. Write this custom payload to file
4. Read this file to check for equivalence

__Notes__
 - Times chosen below are arbitrary for sake of example, variable names containing `utime` reflect the time of an event in UTC, since the start of the year, measured in 0.1 ns
 - Scaler counts follow a poisson distribution, `scaler_lambda` was arbitrariy selected
 - Dictionary with keys matching keyword args of construct_payload may be unpacked via **
   - These values can just be passed as arguments to `construct_payload`, I just like using dicts for example code
 

In [3]:
test_file = './data/test.dat'
with SN_PayloadWriter(test_file, overwrite=True) as w:
    ### Test writer against existing payload
    w.write(pay)
    
    ### Test Construction of custom payload
    
    utime = 123456789  # Time of payload collection
    launch_utime = 120006539  # Time of DOM power-on
    n_scalers = 670
    scaler_lambda = 0.8  # 

    
    custom_pay_dict = {
        'utime': utime, 
        'dom_id': 0xe9fed8c717dd,  # DOM [1-6] Chutes_and_Ladders, value equivalent to 257280767891421
        'domclock': (utime - launch_utime)//250,  # Integer number of 25 ns clock cycles since DOM activation
        'scalers': np.random.poisson(scaler_lambda, size=n_scalers),
        'keep_data': True
    }
    
    custom_payload = construct_payload(**custom_pay_dict)
    
    ### Test writer against custom payload
    w.write(custom_payload)

#### Sanity check via print(payload) before reading from file

In [4]:
print('=== Expected Custom Payload __str__ (Sanity check only, Not a test of equivalence) ===')
print("Supernova@{0:d}[dom {1:012x} clk {2:012x} scalerData*{3:d}]\n".format(
    utime, custom_pay_dict['dom_id'], custom_pay_dict['domclock'], n_scalers))
print(custom_payload)

=== Expected Custom Payload __str__ (Sanity check only, Not a test of equivalence) ===
Supernova@123456789[dom e9fed8c717dd clk 0000000035e9 scalerData*670]

Supernova@123456789[dom e9fed8c717dd clk 0000000035e9 scalerData*670


#### Sanity check via print(payload) after reading from file

In [5]:
with SN_PayloadReader(test_file) as r:
    while r.nrec < 2:
        pay = next(r)
        print(pay)
        
print('\n=== Expected Custom Payload __str__ (Sanity check only, Not a test of equivalence) ===')
print("Supernova@{0:d}[dom {1:012x} clk {2:012x} scalerData*{3:d}]".format(
    utime, custom_pay_dict['dom_id'], custom_pay_dict['domclock'], n_scalers))

Supernova@157144851073427008[dom 88ce18beda45 clk 03103a410001 scalerData*652
Supernova@123456789[dom e9fed8c717dd clk 0000000035e9 scalerData*670

=== Expected Custom Payload __str__ (Sanity check only, Not a test of equivalence) ===
Supernova@123456789[dom e9fed8c717dd clk 0000000035e9 scalerData*670]


## Creating sndata for Unit-testing

Real sndata.dat files are constructed taking into consideration a few principles
1. Every payload in a .dat file is arranged in ascending time, and every .dat file is also ordered in ascending times
2. Every sequential payload for a particular DOM has a utime that corresponds to the scaler readout immediately following the last scaler in the previous file.
 - Scalers are collected over a period of $2^{16}$ clock cycles, so in units 0.1 ns, that is $2^{16} \times 250 = 16384000$, or 1.6384 ms.
 - In brief, each scaler is collected 1.6384 ms after the previous scaler.
 - The payload `utime` and `domclock` fields refers to the time and no. of DOM clock cycles at the first scaler in the payload
 - For DOM $i$ with first payload $p_{i,0}$, `utime` $t_{i,0}$ and $n_{i,0}$ scalers, the second payload for that DOM in the file $p_{i,1}$ will have `utime` $t_{i,1} = t_{i,0} + n_{i,0} \times (2^{16} \times 250)$
3. Real sndata files collect ~190 MB of data before a new file is created
4. Every DOM in the run configuration issues payload $n$ before any DOM issues paylaod $n+1$

Committing large files is not recommended, but having some files would be useful for unit testing.

A data set would look something like the following

 - Small size (<25MB). 
   - File size is prop. to duration and number of DOMs. Having a longer duration is more important for testing, so a test set might only need one or several strings of the detector. 
 - Adheres to the file format/principles above
 - Contains specific features (Unless it is intended to represent background) that can affect trigger significance, to test trigger formation algorithm
   - Occasionally in real data, the one or more of the conditions will not be met, and SNDAQ will handle it somewhat quietly (usually) For the sake of testing, we should assume all of these conditions to be met, unless we are making a test set to specifically examine how SNDAQ handles these circumstances.

In [2]:
from sndaq.detector import Detector

In [3]:
i3 = Detector()

In [20]:
# Random valid ids
np.random.seed(42)  # Randomly selected results for the seed 42, this allows the selection to be reproduced
valid_ids = np.random.choice(i3.dom_table['mbid'], size = 10)
dict_valid_ids = {'doms': [mbid for mbid in valid_ids]} 
print(dict_valid_ids)

{'doms': [221612503003558, 261790137316497, 220496300680470, 170083391137477, 207507749111472, 59697474087023, 113721244932466, 280538975565061, 133055326516044, 25503628325740]}


In [35]:
# Introduce some invalid ids (by modifying valid ids)
np.random.seed(43)
mask = np.random.randint(0,2,size=valid_ids.shape).astype(np.bool)
mask = np.invert(mask)

mixed_ids = np.array(valid_ids)  # This creates a seperate instance of the variable valid_ids
mixed_ids[mask] += 4

dict_mixed_ids = {'doms': [mbid for mbid in mixed_ids]} 
print(dict_mixed_ids)

print('\nModified Ids:')
print('{0:5s} | {1:>15s} | {2:>15s}'.format('Index', 'Valid ID', 'Invalid ID'))
for idx, valid_mbid, invalid_mbid in zip(mask.nonzero()[0], valid_ids[mask], mixed_ids[mask]):
    print(f'{idx:>5d} |{valid_mbid:>16d} |{invalid_mbid:>16d}')

{'doms': [221612503003614, 261790137316553, 220496300680470, 170083391137477, 207507749111472, 59697474087079, 113721244932522, 280538975565061, 133055326516044, 25503628325740]}

Modified Ids:
Index |        Valid ID |      Invalid ID
    0 | 221612503003610 | 221612503003614
    1 | 261790137316549 | 261790137316553
    5 |  59697474087075 |  59697474087079
    6 | 113721244932518 | 113721244932522


In [46]:
# Random valid strings
np.random.seed(48)  
valid_str = np.random.randint(1,86, size=2)
dict_valid_str = {'str': [mbid for mbid in valid_str]} 
# 'doms': [mbid for mbid in valid_ids],

{'str': [1, 52]}


In [47]:
# Introduce some invalid strings (by modifying valid ids)
mixed_str = np.array(valid_str)
mixed_str[-1] = 99
dict_mixed_str = {'str': [mbid for mbid in mixed_str]} 
print(dict_mixed_str)

{'str': [1, 99]}


In [48]:
# Account for duplicate entries on strings and DOM ids
np.random.seed(42)  # Randomly selected results for the seed 42, this allows the selection to be reproduced
dom_str = 1
valid_ids = np.random.choice(i3.dom_table['mbid'], size = 10)
str_ids = np.random.choice(i3.dom_table['mbid'][i3.dom_table['str']==dom_str], size = 10)
valid_ids = np.append(valid_ids, str_ids)
dict_duplicates = {
    'doms': [mbid for mbid in valid_ids],
    'str': [1]
}
print(dict_duplicates)

{'doms': [221612503003558, 261790137316497, 220496300680470, 170083391137477, 207507749111472, 59697474087023, 113721244932466, 280538975565061, 133055326516044, 25503628325740, 207271527053912, 20579555797555, 76224551397868, 167080981203282, 31155393599125, 171972502366323, 20579555797555, 138631493291764, 266169248464026, 125366650734220], 'str': [1]}
